In [49]:
# public packages
import andes
import ams
import cvxpy as cp
import numpy as np
import gurobipy

# local packages
import anci_fun as af

# Para preparetion

In [50]:
# neural network parameters 
nn_name = 'trained_model.pth'
nn_weights, nn_bias, nn_diam = af.extract_nn_para(nn_name) # three list

# nn_diam = [27, 512, 6]

In [51]:
# MD parameters
vsg_num = 8

lb_M = np.array([0, 0, 0, 0, 0, 0, 0, 0])
ub_M = np.array([6, 6, 6, 6, 6, 6, 6, 6])

lb_D = np.array([0, 0, 0, 0, 0, 0, 0, 0])
ub_D = np.array([4, 4, 4, 4, 4, 4, 4, 4])

In [52]:
sf_load = 1.0

# y = [rocof_max, fnadir, dtheta_max, busV_min, eig_max, eig_max_]
lb_y = np.array([-0.5, -0.6, -3.14, 0, -100, -100])
ub_y = np.array([ 0.5,  0.6,  3.14, 2,    0,    0])

# Load ams model

In [53]:
ssp = ams.load('ams_case123_REGCV1_v5.xlsx')

ssp.LDOPF.run()

ssp.LDOPF.om.mdl.solve()

ssp.LDOPF.om.vars

LDOPF solved as optimal in 0.0600 seconds with exit code 0.


OrderedDict([('pg', Variable((11,))),
             ('pn', Variable((117,))),
             ('qg', Variable((11,))),
             ('qn', Variable((117,))),
             ('vsq', Variable((117,))),
             ('pl', Variable((116,))),
             ('ql', Variable((116,)))])

In [54]:
ssp.LDOPF.om.pg.value

array([0.02000001, 0.00000001, 0.00000002, 0.05448514, 0.07447952,
       0.03400588, 0.03400588, 0.03400588, 0.03400588, 0.03400588,
       0.03400588])

In [55]:
ssp.LDOPF.pg.get_idx()

['PV_1',
 'PV_10',
 'PV_11',
 'PV_2',
 'PV_3',
 'PV_4',
 'PV_5',
 'PV_6',
 'PV_7',
 'PV_8',
 'PV_9']

# Define new variables, constraints, and obj

In [56]:
# M and D

M = cp.Variable(vsg_num, name="M")
D = cp.Variable(vsg_num, name="D")

constraints_M = [M >= lb_M, M <= ub_M]
constraints_D = [D >= lb_D, D <= ub_D]


In [57]:
# data for the neural network linearization
pg    = ssp.LDOPF.om.pg

x1    = cp.hstack([pg, M, D])  # this version doesn't incelu load scaling factor

w1    = nn_weights[0]
w2    = nn_weights[1]

b1    = nn_bias[0]
b2    = nn_bias[1]

hdown = -100
hup   = 100


In [58]:
# neural network linization

z = cp.Variable(nn_diam[1], name="z")
a = cp.Variable(nn_diam[1], name="a", boolean=True)

z_ = w1 @ x1 + b1

y = w2 @ z + b2

constraints_nn = [
                    z <= z_ - hdown * (1 - a),
                    z >= z_,
                    z <= hup * a,
                    z >= 0,
                    y <= ub_y,
                    y >= lb_y
                ]
                

In [59]:
z

Variable((512,))

In [60]:
constriants_ams = [ssp.LDOPF.om.constrs[cname] for cname in ssp.LDOPF.om.constrs.keys()]

In [61]:
# constraints = constraints_M + constraints_D + constraints_nn + constriants_ams

constraints = constriants_ams + constraints_M + constraints_D

# Formulate new problem

pg = ['PV_1',
 'PV_10',
 'PV_11',
 'PV_2',
 'PV_3',
 'PV_4',
 'PV_5',
 'PV_6',
 'PV_7',
 'PV_8',
 'PV_9']

 SG: PV_1, PV_2, PV_3 (slack bus)

 IBR: PV_4, PV_5, ..., PV_11

In [62]:
c2 = np.array([0.01, 0, 0, 0.01, 0.02, 0, 0, 0, 0, 0, 0])
c1 = np.array([  20, 0, 0,   20,   30, 0, 0, 0, 0, 0, 0])

d  = np.array([0, 0, 0, 0, 1, 0, 1, 0])

pg_power = cp.power(pg, 2)

cost = cp.sum(c2 @ pg_power + c1 @ pg + d @ M)

obj = cp.Minimize(cost)

In [63]:
# Create the optimization problem
prob = cp.Problem(obj, constraints)

# Solve the optimization problem
prob.solve(solver=cp.GUROBI, verbose=True)
# prob.solve()

                                     CVXPY                                     
                                     v1.3.2                                    
(CVXPY) Sep 06 12:02:51 AM: Your problem has 621 variables, 17 constraints, and 0 parameters.
(CVXPY) Sep 06 12:02:51 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Sep 06 12:02:51 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Sep 06 12:02:51 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Sep 06 12:02:51 AM: Compiling problem (target solver=GUROBI).
(CVXPY) Sep 06 12:02:51 AM: Reduction chain: CvxAttr2Constr -> Qp2SymbolicQp -> QpMatrixStuf

SolverError: Solver 'GUROBI' failed. Try another solver, or solve with verbose=True for more information.

# 